# LLM CAPABILITIES IN PREDICTION TASKS

## LOAD CLIENT

In [13]:
import json
from ollama import Client

# Initialize the client
client = Client(host='http://localhost:11434', timeout=30)

In [3]:
def generate_response(prompt):
    """
    Generates a response using the Ollama API.
    """
    response = client.generate( model='mistral',
                prompt = prompt,
                format="json")
    return json.loads(response["response"])


def view_response(prompt):
    """
    Views a response using the Ollama API, in a structured format.
    """
    response = generate_response(prompt)
    print(json.dumps(response, indent=2))

## PROMPT ENGINEERING

### Designing a simple System prompt and just giving the response type

In [ ]:
SYSTEM_PROMPT = (
    "You are an expert superforecaster, familiar with the work of Tetlock and others. "
)
def base_prompt(query):
    prompt = (
        SYSTEM_PROMPT
        + "\n\n"
        + "QUESTION: " + query
        + "\n\n"
        + "Return a structured response in JSON format"
        + "containing the probability of the yes/no response to the question and a"
        + "measure of confidence between 0 and 1."
    )
    return (
        prompt
    )

In [ ]:
prompt = base_prompt("Will the Liberal Party win the most seats in the 2021 Canadian federal election?")
view_response(prompt)

{
  "question": "Will the Liberal Party win the most seats in the 2021 Canadian federal election?",
  "answer": {
    "yes": 0.6,
    "no": 0.4,
    "uncertainty": 0.2,
    "confidence": 0.9
  },
  "explanation": "Based on current polling trends and historical voting patterns, the Liberal Party has a 60% chance (0.6) of winning the most seats in the 2021 Canadian federal election. The opposing Conservative Party has a 40% chance (0.4). This analysis is based on available data up to the current date and assumes no significant changes in public opinion or events between now and the election. The uncertainty factor is set at 10%, indicating a moderate level of confidence in this forecast."
}


In [ ]:
prompt = base_prompt("Will the Liberal Party win the most seats in the 2021 Canadian federal election?")
view_response(prompt)

{
  "question": "Will the Liberal Party win the most seats in the 2021 Canadian federal election?",
  "answer": {
    "yes": 0.65,
    "no": 0.35
  },
  "confidence": 0.8,
  "explanation": "Based on current polling data and historical trends, the Liberal Party has a 65% chance of winning the most seats in the 2021 Canadian federal election. However, elections are subject to uncertainties and unexpected events. The confidence level for this prediction is 80%."
}


In [ ]:
prompt = base_prompt("Will the Liberal Party win the most seats in the 2021 Canadian federal election?")
view_response(prompt)

{
  "question": "Will the Liberal Party win the most seats in the 2021 Canadian federal election?",
  "answer": {
    "yes": 0.6,
    "no": 0.4,
    "confidence": 0.8
  },
  "explanation": "Based on current polling data and historical voting trends, there is a 60% chance that the Liberal Party will win the most seats in the 2021 Canadian federal election. The confidence level for this prediction is high at 80%, but elections are inherently uncertain and events beyond our control may influence the outcome."
}


#### NOTE: The responses vary to certain length both in reasoning and confidence. The strcuture of the json changes at times and is not consistent.

### Define the structure of response for reproducability

In [8]:
from queries.constant import Questions

In [11]:
SYSTEM_PROMPT = (
    "You are an expert superforecaster, familiar with the work of Tetlock and others. "
)


def base_prompt(query):
    prompt = (
        SYSTEM_PROMPT
        + "\n\n"
        + "QUESTION: " + query
        + "\n\n"
        + "Return a structured response in JSON format"
        + "containing the probability of the yes/no response to the question and a"
        + "measure of confidence between 0 and 1. Provide output in JSON format as follows:"
        + """{"answer":{"yes": ..., "no": ...}, "confidence": ..., "explanation": ...}"""
    )
    return (
        prompt
    )


def reproducability_test(prompt):
    """
    Tests the reproducibility and consistency of the response for a prompt.
    """
    for i in range(0, 3):
        prompt = base_prompt(prompt)
        print("TRIAL " + str(i))
        view_response(prompt)

In [14]:
prompt = base_prompt(Questions.QUESTION_1)
view_response(prompt)

{
  "answer": {
    "yes": 0.55,
    "no": 0.45
  },
  "confidence": 0.8,
  "explanation": "Based on current polling trends and historical voting patterns, the Liberal Party has a slightly higher chance of winning the most seats in the 2021 Canadian federal election than the Conservative Party. However, there is still significant uncertainty due to various factors such as campaign performance, unexpected events, and voter turnout. Therefore, our confidence level is relatively low but we lean towards the Liberal Party having a edge in terms of seat count."
}


In [15]:
prompt = base_prompt(Questions.QUESTION_2)
view_response(prompt)

{
  "answer": {
    "yes": 0.25,
    "no": 0.75
  },
  "confidence": 0.9,
  "explanation": "At this point in time, the odds of Armin Laschet becoming the next chancellor of Germany in 2022 are estimated to be around 25%. The Christian Democratic Union (CDU) leader is considered a strong contender, but several factors can influence the outcome of the German federal elections, such as campaign strategies, alliances, and unforeseen events. The current political landscape is also subject to change. Therefore, while there is a 25% chance that Armin Laschet will be the next chancellor, there is still uncertainty, which is reflected in a confidence level of 90%."
}


#### REPRODUCABILITY ANALYSIS

In [16]:
reproducability_test(Questions.QUESTION_1)

TRIAL 0
{
  "answer": {
    "yes": 0.65,
    "no": 0.35
  },
  "confidence": 0.8,
  "explanation": "Based on current polling trends and historical voting patterns, the Liberal Party has a slightly better than average chance of winning the most seats in the 2021 Canadian federal election. However, elections can be influenced by many unforeseen factors, so the confidence level is not high."
}
TRIAL 1
{
  "answer": {
    "yes": 0.65,
    "no": 0.35
  },
  "confidence": 0.8,
  "explanation": "Based on current polling data and historical trends, the Liberal Party has a 65% chance of winning the most seats in the 2021 Canadian federal election. However, elections can be influenced by many unpredictable factors, so this prediction comes with a confidence level of 80%. The Liberal Party's strong performance in recent years and the leadership of Justin Trudeau make them a favorite, but there are still risks and uncertainties that could lead to a different outcome."
}
TRIAL 2
{
  "answer": {
   

In [17]:
reproducability_test(Questions.QUESTION_2)

TRIAL 0
{
  "answer": {
    "yes": 0.25,
    "no": 0.75
  },
  "confidence": 0.9,
  "explanation": "Armin Laschet is one of the leading candidates for the German Chancellorship in 2022, but there are several other strong contenders including Olaf Scholz and Annalena Baerbock. At this point, the odds seem to slightly favor a 'no' response due to the uncertainty surrounding the outcome of the political situation. However, given the volatility of elections and the possibility of unexpected events, our confidence in this prediction is relatively low."
}
TRIAL 1
{
  "answer": {
    "yes": 0.25,
    "no": 0.75
  },
  "confidence": 0.8,
  "explanation": "According to current polling data and political trends, Armin Laschet is considered a strong contender for the German Chancellorship in 2022. However, there are several factors that could influence the outcome of the election such as unexpected events or shifts in public opinion. Based on available information, I estimate the probability of h

In [24]:
reproducability_test(Questions.QUESTION_3)

TRIAL 0
{
  "answer": {
    "yes": 0.95,
    "no": 0.05
  },
  "confidence": 0.9,
  "explanation": "Based on current climate trends and temperature data, it is highly likely (95%) that NASA will record 2023 as one of the hottest years on record. However, there is still a small chance (5%) that another year could surpass it, so the confidence level is set at 90%."
}
TRIAL 1
{
  "answer": {
    "yes": 0.35,
    "no": 0.65
  },
  "confidence": 0.8,
  "explanation": "Based on current trends and climate model projections, it is likely that 2023 will be warmer than previous years due to human-induced greenhouse gas emissions and natural climate variability. However, there are several factors that could influence the global temperature trend in any given year, such as El Ni\u00f1o or La Ni\u00f1a events, volcanic eruptions, and weather patterns. Therefore, while it is plausible that NASA will record 2023 as the hottest year on record, there is still a significant chance (65%) that another yea

In [22]:
reproducability_test(Questions.QUESTION_4)

TRIAL 0
{
  "answer": {
    "yes": 0.6,
    "no": 0.4
  },
  "confidence": 0.7,
  "explanation": "Based on current market trends and analysis of historical data, the probability of Bitcoin's price being above $46,000 on April 14, 2022 is estimated to be 60%. However, it's important to note that the price of Bitcoin is subject to significant volatility and can be influenced by various factors such as regulatory actions, market sentiment, and technological developments. Therefore, the confidence level in this prediction is set at 70%."
}
TRIAL 1
{
  "answer": {
    "yes": 0.55,
    "no": 0.45
  },
  "confidence": 0.7,
  "explanation": "Based on current market trends and analysis, the price of Bitcoin (BTC) has been fluctuating around $46,000 in recent days. Given this, I assess a 55% probability that the price will be above $46,000 on April 14, 2022. However, there is significant uncertainty and volatility in the Bitcoin market, which reduces my confidence in this prediction to 70%."
}
T

#### NOTE: The structure is consitent now. The responses are pretty consistent for politcal questions, that too for the questions that are before cut off date of the model. We can observe in the covid question, the model is having tough time in coming to any conclusion. This can depict the model's incompetency in forecasting based problems which might require a certain amount of domain knowledge or recent data around that time period.



### ADDING CONTEXT WITH DOMAIN KNOWLEDGE TO THE PROMPT TO ANSWER THE QUESTION

In [25]:
def base_prompt(query, context_str):
    prompt = (
        SYSTEM_PROMPT
        + "Context information is below.\n"
        + "---------------------\n"
        + f"{context_str}\n"
        + "---------------------\n"
        + "\n\n"
        + "QUESTION: " + query
        + "\n\n"
        + "Return a structured response in JSON format"
        + "containing the probability of the yes/no response to the question and a"
        + "measure of confidence between 0 and 1. Provide output in JSON format as follows:"
        + """{"answer":{"yes": ..., "no": ...}, "confidence": ..., "explanation": ...}"""
    )
    return (
        prompt
    )

In [27]:
context_str = (
    "The historical trend of weekly covid cases from Nov 13, 2021 to Dec 18, 2021:\n"
    "November 13, 2021: 524,688 cases\n"
    "November 20, 2021: 529,885 cases\n"
    "November 27, 2021: 430,789 cases\n"
    "December 4, 2021: 561,474 cases\n"
    "December 11, 2021: 544,989 cases\n"
)

view_response(base_prompt(Questions.QUESTION_4, context_str))

{
  "answer": {
    "yes": 0.2,
    "no": 0.8
  },
  "confidence": 0.7,
  "explanation": "Based on the available data and expertise in superforecasting, it is currently impossible to accurately predict the price of Bitcoin (BTC) on April 14, 2022 with a high degree of confidence. The historical trend of weekly COVID-19 cases from November 13, 2021 to December 18, 2021 was provided in the context and does not directly relate to the price of Bitcoin. The price of Bitcoin is influenced by various factors including but not limited to market sentiment, regulatory decisions, technological developments, and global economic conditions. A prediction with a probability of 20% for ",
  "yes": " and 80% for ",
  "no": " should be considered as highly uncertain."
}


#### NOTE: Providing context information with historical answer, helped the model to be more accurate and correct.

In [28]:
view_response(base_prompt(Questions.QUESTION_4, context_str))

{
  "answer": {
    "yes": 0.2,
    "no": 0.8
  },
  "confidence": 0.7,
  "explanation": "Based on the available information and expertise in forecasting trends, it is currently uncertain whether the price of Bitcoin will be above $46,000 on April 14, 2022. The historical trend of COVID-19 cases does not provide any relevant information for this question. The price of Bitcoin is influenced by various factors including market sentiment, regulatory environment, and technological developments. As a result, it's difficult to make an accurate prediction without considering these factors and the current state of the Bitcoin market."
}


In [29]:
view_response(base_prompt(Questions.QUESTION_4, context_str))

{
  "answer": {
    "yes": 0.3,
    "no": 0.7
  },
  "confidence": 0.8,
  "explanation": "Based on the available data and expert knowledge in superforecasting, it is not possible to directly predict the future price of Bitcoin with certainty based on historical COVID-19 case trends. However, given current market conditions, a probability of 30% for Bitcoin being above $46,000 on April 14, 2022, and a probability of 70% for it being below that level can be assigned. This assessment is subject to significant uncertainty and should not be considered as definitive."
}


#### NOTE: The responses are also more reproducible now, as the context information about historical pattern of covid helps model in making more accurate responses

In [30]:
context_str = "This is a market on the higher placed finisher at the 2022 Saudi Arabia Grand Prix, Max Verstappen or Charles Leclerc. The race takes place over 50 laps of the 6.174-kilometer Jeddah Corniche Circuit on Sunday, March 27, 2022."

view_response(base_prompt(Questions.QUESTION_6, context_str))

{
  "answer": {
    "yes": 0.55,
    "no": 0.45
  },
  "confidence": 0.8,
  "explanation": "Based on current form and performance data from previous races, Max Verstappen has a slightly better chance of finishing ahead at the Saudi Arabia Grand Prix than Charles Leclerc, with a probability of approximately 55%. However, this prediction is subject to a high degree of uncertainty and could change based on various factors such as car performance, track conditions, and driver mistakes."
}


In [31]:
view_response(base_prompt(Questions.QUESTION_6, context_str))

{
  "answer": {
    "yes": 0.55,
    "no": 0.45
  },
  "confidence": 0.7,
  "explanation": "Both Max Verstappen and Charles Leclerc have shown strong performances in recent Formula 1 races. Max Verstappen is currently the World Champion, and Charles Leclerc has been consistently finishing on the podium as well. The Jeddah Corniche Circuit features long straightaways and high-speed corners, favoring cars with good power and aerodynamics. Both Red Bull Racing (Verstappen) and Ferrari (Leclerc) have competitive cars this season. Based on current form, I give a slightly higher probability to Max Verstappen finishing ahead at the Saudi Arabia Grand Prix. However, there are many variables involved in motorsports, including strategy, weather conditions, and unforeseen incidents that can affect the outcome. Thus, my confidence is moderate."
}


### NOTES
#### The model performs well when context string is provided which help the model in understading the back history of the question. Reproducability of the model is really good at this level, but this clearly demosntrates that for such type of question building a RAG pipeline which has a module
#### for Information retrieval is really necesaar, especailly for the question that are post the knowledge cut off date.